In [1]:
# **Imports and Constants**

In [58]:
# imports
from __future__ import print_function
from pyspark import SparkContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from mpl_toolkits.mplot3d import Axes3D
from pyspark.sql.functions import col
from functools import reduce
from pyspark import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, IntegerType
from pyspark.sql.functions import col, split, size, isnan, array_contains, array_min, when, count
from pyspark.sql.types import StructType, StructField, TimestampType, StringType, FloatType
import pyspark.sql.functions as f
import matplotlib.pyplot as plt
import pandas as pd
import glob
import re
import pathlib
import json
import datetime as dt
import numpy as np
import os
import random
import statistics
import platform

In [3]:
# paths
DATASET_PATH = '/Users/Soroush/Desktop/Thesis/Code/dataset/38_CER Electricity_Gas/CER Electricity Revised March 2012'

# BASE_PATH=pathlib.Path().absolute()
# KMEANS_REL_PATH="kmeans models"
# DATASET_REL_PATH="dataset"
# DATASET_PATH=os.path.join(BASE_PATH,DATASET_REL_PATH)
# KMEANS_PATH=os.path.join(BASE_PATH,KMEANS_REL_PATH)

# from google.colab import drive
# drive.mount('/gdrive')

In [4]:
# env variables
if platform.system() == 'Windows':
    %env PYSPARK_DRIVER_PYTHON = python
    %env PYSPARK_PYTHON = python
elif platform.system() == 'Linux':
    %env PYSPARK_DRIVER_PYTHON = python
    %env PYSPARK_PYTHON = python3
else:
    %env PYSPARK_DRIVER_PYTHON = python3.6
    %env PYSPARK_PYTHON = python3.6

# incompatibility with Pyarrow
# need to install Pyarrow 0.14.1 or lower or Set the environment variable ARROW_PRE_0_15_IPC_FORMAT=1
%env ARROW_PRE_0_15_IPC_FORMAT = 1

env: PYSPARK_DRIVER_PYTHON=python3.6
env: PYSPARK_PYTHON=python3.6
env: ARROW_PRE_0_15_IPC_FORMAT=1


In [5]:
# load and save .read_pickle() and .to_pickle()

# save
# dataset.to_pickle(DATASET_PATH+"dataset.pkl")
# aggregated_dataset.to_pickle(DATASET_PATH+"aggregated_dataset.pkl")
# json_dataset.to_pickle(DATASET_PATH+"json_dataset.pkl")
# dataset.to_csv(DATASET_PATH+"dataset.csv")
# aggregated_dataset.to_csv(DATASET_PATH+"aggregated_dataset.csv")
# json_dataset.to_csv(DATASET_PATH+"json_dataset.csv")
# aggregated_dataset_rowBased.to_csv(DATASET_PATH+"aggregated_dataset_rowBased.csv")


# load
# dataset=pd.read_pickle(DATASET_PATH+"dataset.pkl")
# aggregated_dataset=pd.read_pickle(DATASET_PATH+"aggregated_dataset.pkl")
# json_dataset=pd.read_pickle(DATASET_PATH+"json_dataset.pkl")

In [49]:
# create appropriate dataset
# load


def load_irish_dataset(dataset_path):

    #data_column_names = ["id", "date", "power"]
    #all_data = pd.DataFrame(columns = data_column_names)
    
    all_data = pd.DataFrame({'id': pd.Series([], dtype='int'),
                   'date': pd.Series([], dtype='int'),
                   'power': pd.Series([], dtype='float')})
    
    files = glob.glob(dataset_path + "/File*.txt") 
    for file in files:
        #print(file)
        data = pd.read_csv(file, sep=" ", header=None)
        data.columns = ["id", "date", "power"]
        #print(data.info())
        all_data = all_data.append(data, ignore_index=True)
    

    # save
    return all_data

dataset = load_irish_dataset(DATASET_PATH)
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157992996 entries, 0 to 157992995
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   id      int64  
 1   date    int64  
 2   power   float64
dtypes: float64(1), int64(2)
memory usage: 3.5 GB
None


In [50]:
#dataset['id'] = dataset['id'].apply(np.int64)
print(dataset.info())
dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157992996 entries, 0 to 157992995
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   id      int64  
 1   date    int64  
 2   power   float64
dtypes: float64(1), int64(2)
memory usage: 3.5 GB
None


,id,date,power
0,2113,19630,0.038
1,2113,19631,0.125
2,2113,19501,0.189
3,2113,19502,0.139
4,2113,19503,0.149


In [60]:
# date column


def create_datetime_column(dataset):

    def code_to_datetime(code):
        output = list(map(int, str(code)))
        day_code = int(output[0]*100+output[1]*10+output[2])
        time_code = int(output[3]*10+output[4])
        # unixtime in 1 January 2009 00:00:00 is 1230768000
        unixtime = 1230768000+(day_code-1)*86400+time_code*1800
        #dt = time.ctime(unixtime)
        out=dt.datetime.utcfromtimestamp(unixtime).strftime('%Y-%m-%d %H:%M:%S')
        return out

    # code_to_datetime(19941)
    dataset['date']=dataset['date'].apply(code_to_datetime)
    dataset['date']= pd.to_datetime(dataset['date']) 
    return dataset


dataset = create_datetime_column(dataset)
print(dataset_with_unixtime.info())
dataset_with_unixtime.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   id      20 non-null     int64         
 1   date    20 non-null     datetime64[ns]
 2   power   20 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 608.0 bytes
None


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,date,power
0,2113,2009-07-15 15:00:00,0.038
1,2113,2009-07-15 15:30:00,0.125
2,2113,2009-07-14 00:30:00,0.189
3,2113,2009-07-14 01:00:00,0.139
4,2113,2009-07-14 01:30:00,0.149


In [37]:
dataset_with_unixtime.head(5)

,id,date_code,power,meter_id,date
0,NaN,19630,0.038,2113.0,Wed Jul 15 19:30:00 2009
1,NaN,19631,0.125,2113.0,Wed Jul 15 20:00:00 2009
2,NaN,19501,0.189,2113.0,Tue Jul 14 05:00:00 2009
3,NaN,19502,0.139,2113.0,Tue Jul 14 05:30:00 2009
4,NaN,19503,0.149,2113.0,Tue Jul 14 06:00:00 2009
